# INDRA baseline

__Goal:__ Evaluate accuracy of INDRA models

__Method:__ Test of INDRA Covid-19 model:
1. Convert Fraunhofer COVID19 knowledge graph to INDRA statements
2. Compare relations with relations from INDRA
4. Calculate accuracy
5. run error analysis


__Data:__ covid-19-kg dataset, [Covid-19 model from INDRA](https://emmaa.indra.bio/dashboard/covid19?tab=model)

__Tools:__ [INDRA](http://www.indra.bio/), [PyBEL](https://github.com/pybel/pybel), [pyobo](https://github.com/pyobo/pyobo)

__Result:__ INDRA models accuracy, results of error analysis

In [ ]:
import pybel
import requests
from indra.processors import bel
from indra.util import batch_iter
from indra.sources import indra_db_rest

In [ ]:
#load graph pre-procesed by Charlie Hoyt: https://github.com/CoronaWhy/bel4corona/tree/master/data/covid19kg
url = 'https://github.com/CoronaWhy/bel4corona/raw/master/data/covid19kg/covid19-fraunhofer-grounded.bel.nodelink.json'
res = requests.get(url)
graph = pybel.from_nodelink(res.json())

In [ ]:

# Process the PyBEL graph into INDRA Statements
pybel_proc = bel.process_pybel_graph(pybel_graph)

# Note that of the ~4k statements in the PyBEL graph, only 831 are successfully
# converted to INDRA statements in large part because of issues with namespaces
covid_stmts = pybel_proc.statements
stmt_hashes = [s.get_hash() for s in covid_stmts]

# Use the INDRA Database REST API client to search for corresponding evidences
# for 100 statements at a time
db_stmts = []
for hash_batch in batch_iter(stmt_hashes, 100):
    idrp = idr.get_statements_by_hash(stmt_hashes, ev_limit=1000)
    db_stmts.extend(idrp.statements)